# Flow ML Course

## TP 3 : Artifacts

L'objectif de ce notebook est d'apprendre à utiliser les artifacts sur Vertex AI

Chaque pipeline créé à l'aide de Vertex AI Pipelines comporte plusieurs `artifacts` et paramètres associés, tels que des modèles, des ensembles de données, des modèles de pipeline et des composants. L'`artifact lineage` de pipeline comprend les facteurs qui ont contribué à sa création, ainsi que les artefacts et les métadonnées associées.

En savoir plus [Artifact Lineage](https://cloud.google.com/vertex-ai/docs/pipelines/lineage#:~:text=The%20lineage%20of%20a%20pipeline%20artifact%20includes%20the,and%20evaluation%20data%20used%20to%20create%20the%20model)

In [4]:
PROJECT_ID = "projet-ia-448520" # "[your-project-id]"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

BUCKET_URI = "gs://cours1bucket" # f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

SERVICE_ACCOUNT = "837843744498-compute@developer.gserviceaccount.com" # "[your-service-account]"  # @param {type:"string"}

In [ ]:
import os
import uuid

import google.cloud.aiplatform as aiplatform

aiplatform.init(project=PROJECT_ID, location=LOCATION, staging_bucket=BUCKET_URI)

## Créer une experiment pour faire le suivi des méta-données

Créer une experiment avec `init()` puis lancer une run dans l'experiment via `start_run()`.

- `aiplatform.init()` - Créer une instance experiment
- `aiplatform.start_run()` - Lancer une run spécifique dans l'experiment
- `aiplatform.end_run()` - Termine une run. Nécessaire pour réaliser d'autres actions dans l'experiment

### Artifact Lineage

Créeons un lineage pour une experiment fictive

In [ ]:
# Create experiment
your_name = ...
EXPERIMENT_NAME = f"supaero-vertex-course-1-3-{your_name}"
aiplatform.init(experiment=EXPERIMENT_NAME)
aiplatform.start_run("run-1")

### Dataset & Model Artifcats

Créons des artifacts synthétiques pour simuler un lineage :

- `dataset_artifact`: Dataset d'input de l'Experiment Run 
- `model_artifact`: Model d'output de l'Experiment Run 

In [27]:
DATASET_URI = "gs://example/dataset.csv"
MODEL_URI = "gs://example/saved_model.pb"

dataset_artifact = aiplatform.Artifact.create(
    schema_title="system.Dataset", display_name="example_dataset", uri=DATASET_URI
)

model_artifact = aiplatform.Artifact.create(
    schema_title="system.Model", display_name="example_modl", uri=MODEL_URI
)

## Artifact lineage

Procédure :
- Initialiser l'éxécution : `start_execution()`
- Attacher à la run l'input `assign_input_artifacts()` 
- Attacher à la run l'output `assign_output_artifacts()` 


In [28]:
with aiplatform.start_execution(
    schema_title="system.ContainerExecution", display_name="example_training"
) as execution:
    execution.assign_input_artifacts([dataset_artifact])

    # Log the params you expect
    ...
    
    # Log the metrics you expect
    ...

    execution.assign_output_artifacts([model_artifact])

    aiplatform.log_metrics(
        {"lineage": execution.get_output_artifacts()[0].lineage_console_uri}
    )

## Extraire les résultats d'un Experiment

Pour terminer une expirement : utiliser `end_run()` 

Ensuite, vous pouvez récupérer la plupart des informations de l'experiment via : `get_experiment_df()` 


In [ ]:
aiplatform.end_run()

experiment_df = aiplatform.get_experiment_df()
experiment_df = experiment_df[experiment_df.experiment_name == EXPERIMENT_NAME]
print(experiment_df.T)

## Visualiser l'Artifact Lineage

Ouvrez le lien suivant pour suivre l'artifcat lineage

In [ ]:
print(
    "Open the following link:", execution.get_output_artifacts()[0].lineage_console_uri
)

#### Supprimer l'Artifact lineage


In [ ]:
try:
    dataset_artifact.delete()
except Exception as e:
    print(e)
try:
    model_artifact.delete()
except Exception as e:
    print(e)

#### Supprimer l'Experiment


In [ ]:
try:
    exp.delete()
except Exception as e:
    print(e)